In [70]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline
from denn import *
import pandas as pd
import seaborn as sns
import io 

In [71]:
nn_p = 5
nn_w = 5
nn_tw = 1
freq = 10
m='mof'

df = get_data(m)
df = df[(df['nnw']==str(nn_w))&(df['nntw']==str(nn_tw))&(df['nnp']==str(nn_p))|(df['isnn']==False)] 

# Plots

In [72]:
df_pivot = get_heatmap_data(m,df)[labels_order]
df_pivot_rank = df_pivot.rank(axis=1)
df_pivot = df_pivot.reset_index()
df_pivot.experiment = df_pivot.experiment.str.slice(3)
df_pivot.set_index(['experiment','function','freq'], inplace=True)

In [73]:
df_pivot_rank = df_pivot_rank.query(f'freq=={freq}')
df_pivot = df_pivot.query(f'freq=={freq}')

In [74]:
df_pivot.style.background_gradient(cmap=plt.cm.Greens, axis=1)

In [75]:
df_pivot_rank.style.background_gradient(cmap=plt.cm.Greens, axis=1)

In [76]:
summary = df_pivot.groupby(['experiment', 'function']).mean()
summary.round(2).style.background_gradient(cmap=plt.cm.Greens, axis=1)

In [77]:
out = summary.round(2).style
for i in range(len(labels_order)//2): 
    out = out.highlight_min(axis=1, subset=labels_order[i*2:(i+1)*2])

out

In [80]:
out.to_excel(PATH_RESULTS/'test.xlsx')

# Interactive

In [ ]:
from ipywidgets import interact
def show_table(dm):
    out = summary[[f'noNN_{dm}', f'NN_{dm}']].round(2)
    return out.style.background_gradient(cmap=plt.cm.Blues_r, axis=1)
    
interact(show_table, dm=['RI', 'HMu','No','Rst'])